# Project of Reinforcement Learning: Stupid Vautour

Authors: Clément Côme, Valentin Gatignol, Nathan De Carvalho

In [ ]:
import gym
import numpy as np
import random
from gym import spaces
from gym.utils import seeding
import math
from scipy.special import comb

## 1) Objectives of the project

### 1.1) Rules of the game

Stupide Vautour is a turn by turn game.

1. Every player is given 15 cards from 1 to 15. A player can see only its cards and can play each of them only once.
2. A deck of 5 malus cards (-5 to -1) and 10 bonus cards (1 to 10) is shuffled.
3. For each card in the deck:
   1. Turn over the bonus/malus card
   2. Each player chooses a card from its hand that he want to play during this turn and put it face down so no one sees it.
   3. When every player has put down its card, they can turn it face up simultaneously.
   4. - If the card is a malus the player that played the smallest card gets it.
      - If the card is a bonus the player that played the highest card gets it.
      - In case of a tie, the card goes to the next smallest (or highest) card.
      - If all players paly the same card, it is discarded and not one gets the card.
4. The final score of each player is computed with the sum of bonuses minus the sum of maluses.
5. The winner is the player with the highest final score

The real rules have other slight details that are not considered here.

### 1.2) Implementing the game

Implementation of "Stupide Vautour" and artificial players.

The project is divided in several parts:

- `implementation/game.py` defines the game rules and flow, an example of its use is given at the end of the file
- `implementation/player.py` defines the players that can be used to play the game
- `implementation/analyse.py` contains several functions to process the results of the games played

### 1.3) Apply Reinforcement Learning Principles to "win the game"

***Intuitive preliminary comments:***

When discussing intuitively what factors should help a "learning" player to find an optimal strategy to beat his adversaries, we point out several issues including:
 - The number of players and the definition of the associated state space
 - The nature of the adversaries: what kind of strategies are they playing?
 - The learning process being *on-line* (i.e. the player learns while playing) or *off-line* (i.e. the player learns by observing people playing and then can play and use his off-line trained strategy)


***A priori objectives of the implementation:***

- Study the game thoroughly for the cases with 2 and 3 players: State space, a priori good and bad strategies.

- Implement several types of artificial players and comment on the game's results.

- Given 1 (or 2) adversaries with fixed strategies, implemented RL algorithms to compute the value function (when the state space is not too large) or to estimate it (when the state space is very large).

- (If enough time) Given a set of strategies, 1 (or 2) adversaries with fixed strategies, choose the strategy that makes us beat the adversaries (apply bandits-based RL principles).

## 2) Definition of the State Space and some ad hoc strategies

### 2.1) Possible actions and State space definitions


#### A reducing set of actions

The number of actions each player can take decreases by one at each turn as he uses one card so that the aim is to find the best moments when to play a card given the state of the game.

For a given number of players $N$, we will define two types of State Spaces (SS) from which the player will have to play a card which remains in his hand - both being exposed to the curse of dimentionality:

#### No-Memory State Space (NMSS): *easy case*

Assume a naive no-memory player only takes into account:

- The current deck's card
- His remaining cards

Such a NMSS **does not depend on the number of players** and this approach will allow to scale easily RL methods to games with numerous players.

Yet, such NMSS is already very large:

$$
|NMSS| = \text{Number of deck's cards}*\sum_{\text{Nber remaining cards} = 1}^{\text{Nber of possible cards}} \binom{\text{Nber of possible cards}}{\text{Nber remaining cards}}
$$

Which is $491505$ possible states for the orignal game with $15$ cards.

*Remark:* we could simplify the game to get reasonable size for the state pace: e.g. for N_cards = 6, we get 378 states so that we can avoid approximating the value function for that case.


#### Machine State Space (MSS): *difficult case*

We consider all the information available to the player, which includes:
- The current deck's card
- The remaining deck's cards

- His remaining cards
- His current score
- The other players' current scores
- The other players' remaining cards

Clearly, **the MSS depends on the number of players and its dimensionality explodes when considering all these information** - and we will have no choice but approximating the value function to deal with such SS.

In [ ]:
# Number of states in NMSS

#V1

Nber_states = 6*(2**6-1)

#V2

N_cards = 6

combinaisons = np.zeros(N_cards)
for i in range(N_cards):
    combinaisons[i] = comb(N_cards,i+1)
Nber_states = N_cards*np.sum(combinaisons) 

print('The number of states for the NMSS is: ', Nber_states, ' with N_cards = ', N_cards)


The number of states for the NMSS is:  378.0  with N_cards =  6


### 2.2) Some proposed strategies

**Very basic strategies - action independent of the deck's card**

- *Random player:* plays a card randomly with a distribution independent of the game's information
- *Max card player:* always plays his maximum card whatever the deck's card
- *Min card player:* always plays his min card whatever the deck's card

**Intermediate strategies - oberve only the deck's card to decide action**

- *Bonus craving player:* plays with high probability a high card to gain high bonuses, and plays randomly for the rest
- *Malus adverse player:* plays with high probability high cards for maluses, and plays randomly for the rest
- *GetRidOfBadCards player:* plays with high probability low cards on small maluses or bonuses and randomly for the rest



In [ ]:
from game import *

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=c3570100-157e-4a3f-b5c9-ebc0870008b3' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>